## 1. 패키지 로드

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

import pandas as pd

from tqdm import tqdm_notebook
import tqdm

## 2. KoGPT2 로드

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

## 3. 학습 데이터 로드

In [ ]:
### 학습 데이터 로드

## 4. 토큰화

In [ ]:
# params setting
batch_size = 32

In [ ]:
# 데이터 로드 및 토큰화 함수
def get_emo_data():
    for question, answer in zip(train_data.감정.to_list(), train_data.문장.to_list()):
        bos_token = [tokenizer.bos_token_id]
        eos_token = [tokenizer.eos_token_id]
        sent = tokenizer.encode('<usr>' + question + '<sys>' + answer) 
        yield bos_token + sent + eos_token

In [ ]:
# 모델이 필요로하는 DataSet 생성
dataset = tf.data.Dataset.from_generator(get_emo_data, output_types=tf.int32)

In [ ]:
# 입력데이터의 크기가 가변 일때 같은 크기로 읽을 수 있게 변환
dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

## 5. 모델 학습

In [ ]:
# 1, 옵티마이저 정의
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

# 2. Step 정의
steps = len(train_data) // batch_size + 1

# 3. 에포치 설정(학습횟수)
EPOCHS = 3

In [ ]:
# 4. 모델 학습
for epoch in range(EPOCHS):
    epoch_loss = 0

    for batch in tqdm.tqdm_notebook(dataset, total=steps):
        with tf.GradientTape() as tape:
            result = model(batch, labels=batch)
            loss = result[0]
            batch_loss = tf.reduce_mean(loss)

        grads = tape.gradient(batch_loss, model.trainable_variables)
        adam.apply_gradients(zip(grads, model.trainable_variables))
        epoch_loss += batch_loss / steps

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

## 6. 전체 모델 저장

In [ ]:
torch.save(model, './model/Gen_sent_GPT2_model.pt') # or pth 확장자 사용